### 전처리된 파일 불러오기 

In [1]:
import pandas as pd 
X_train = pd.read_csv('./X_train.csv')
y_train = pd.read_csv('./y_train.csv')
X_test = pd.read_csv('./X_test.csv')


In [2]:
X_train.shape, y_train.shape, X_test.shape

((891, 30), (891, 1), (418, 30))

### 앙상블 모델 사용하기
- 머신러닝에서 성능이 좋은 모델
- 여러개의 DecisionTree 모델을 사용 - RandomForest 모델 

#### RandomForest 모델 불러오기 

In [3]:
from sklearn.ensemble import RandomForestClassifier

In [4]:
# 50개의 DT를 통해서 RandomForest 모델 생성 
forest_model = RandomForestClassifier(n_estimators=50)
forest_model

RandomForestClassifier(n_estimators=50)

In [5]:
import warnings
warnings.filterwarnings('ignore') # warning을 무시하기

In [6]:
# 교차검증 (cv = 5)
from sklearn.model_selection import cross_val_score
# 사용할 모델, 문제 데이터, 정답데이터, 데이터분할수
# 학습
forest_model.fit(X_train, y_train)
pre = forest_model.predict(X_test) # 평가용 문제로 예측데이터를 생성
pre
# 평가 
score = cross_val_score(forest_model, X_train, y_train, cv=5)
score.mean()
# 예측 결과를 csv 파일로 내보내기 
gender_sub = pd.read_csv('./Data/titanic/gender_submission.csv')
gender_sub['Survived'] = pre
gender_sub
# Kaggle에 업로드 후 점수 확인해보기 

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,1
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [7]:
gender_sub.to_csv('mysub03.csv', index=False)

### 하이퍼 파라미터 튜닝

In [8]:
forest_model1 = RandomForestClassifier(n_estimators=100)

In [26]:
# 하이퍼 파라미터 튜닝

param = { 'n_estimators': [50, 70, 90, 110, 130, 150],
        'max_depth' :[3, 4, 5, 6, 7],
       'max_features': [0.3, 0.4, 0.5, 0.6, 0.7] }

# n_estimators = [50, 70, 90, 110, 130, 150]
# max_depth = [3, 4, 5, 6, 7]
# max_features = [0.3, 0.4, 0.5, 0.6, 0.7]



In [10]:
from sklearn.model_selection import GridSearchCV

In [11]:
# GridSearchCV 적용 
# 하이퍼 파라미터를 딕셔너리형태로 저장 (param)
grid = GridSearchCV(RandomForestClassifier(), param, cv=5)
grid

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [3, 4, 5, 6, 7],
                         'max_features': [0.3, 0.4, 0.5, 0.6, 0.7],
                         'n_estimators': [50, 70, 90, 110, 130, 150]})

In [15]:
grid.fit(X_train, y_train)

GridSearchCV(cv=5, estimator=RandomForestClassifier(),
             param_grid={'max_depth': [3, 4, 5, 6, 7],
                         'max_features': [0.3, 0.4, 0.5, 0.6, 0.7],
                         'n_estimators': [50, 70, 90, 110, 130, 150]})

In [13]:
grid.best_params_

{'max_depth': 4, 'max_features': 0.7, 'n_estimators': 70}

In [14]:
grid.best_score_

0.8372481325717155

In [27]:
# 찾아낸 최적의 조합을 적용한 모델로 학습, 평가, 예측 --> csv파일로 내보내기

# Kaggle에서 점수 확인 
tree_model2 = RandomForestClassifier(max_depth= 4, max_features = 0.7, n_estimators = 70)
tree_model2.fit(X_train,y_train)

RandomForestClassifier(max_depth=4, max_features=0.7, n_estimators=70)

In [24]:
# 1. 예측
pre1 = tree_model2.predict(X_test) # 평가용 문제로 예측데이터를 생성
pre1
# 2. 답안지 파일 불러오기 
gender_sub1 = pd.read_csv('./Data/titanic/gender_submission.csv')
# 3. 답안지 파일에 예측값 대입 
gender_sub1['Survived'] = pre1
gender_sub1.to_csv('mysub04.csv', index=False)